In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
import itertools
class Apriori:
    def __init__(self, min_support, min_conf, input_file_name, output_file_name):
        self.min_support = min_support
        self.min_conf = min_conf
        self.input_file_name = input_file_name
        self.output_file_name = output_file_name
        self.DataBase = []
        self.C = []
        
    def Readfile(self):
        fp = open(self.input_file_name, "r")
        line = fp.readline()
        while line:#一行一行讀取
            line_string_list = line.split()
            line_string_map = map(int, line_string_list)#將string list轉成 int map
            line_int_list = list(line_string_map)#將int map轉成 int list
            self.DataBase.append(line_int_list)#加進database中
            #print(line_int_list)
            line = fp.readline()
        fp.close()
    
    def Find_max_in_DataBase(self):#找出DataBase中最大的數
        DataBase_max = 0
        for i in self.DataBase:
            if max(i) > DataBase_max:
                DataBase_max = max(i)
        return DataBase_max
    
    def CreateC1(self):
        C1 = []
        DataBase_max = self.Find_max_in_DataBase()
        #print(f"DataBase_max:{DataBase_max}")
        for i in range(0, DataBase_max+1):
            C1.append([i, 0])#create C1 with appropriate 0
        #print(f"Length of C1:{len(self.C1)}")
        for i in self.DataBase:
            for j in i:
                C1[j][1] = C1[j][1] + 1 #create C1
        self.C.append(C1)
        
    def CtoL(self, C):
        #print(len(C))
        i = 0
        while i < len(C):
            if C[i][1] < self.min_support:
                C.remove(C[i])
            else:
                i = i + 1
        #self.C1 = C.copy()        
        #print(self.C1)
        return C

    def Output(self, L, time, of):
        for i in L:
            sub_itemset = []
            for p in range(time, 1, -1):
                #print(self.find_combinations(i[0], p-1))
                sub_itemset.append(self.find_combinations(i[0], p-1))
            #print(sub_itemset)
            
            for p in range(0, time-1):
                for j in sub_itemset[p]:
                    #print(j)
                    for k in range(len(self.C[time - p - 2])):
                        if len(j) == 1:
                            if self.C[time - p - 2][k][0] == j[0]:
                                #print(k)
                                break
                        else:
                            if self.C[time - p - 2][k][0] == j:
                                #print(k)
                                break
                    if (float(i[1]) / float(self.C[time - p - 2][k][1])) >= (float(self.min_conf)/100):
                        i_tmp = []
                        for g in i[0]:
                            i_tmp.append(g)
                        for g in j:
                            i_tmp.remove(g)
                        if len(j) == 1:
                            print(f"{j[0]} -> ", end = '')
                            of.write(str(j[0]))
                            of.write(" -> ")
                        else:
                            print("{", end = '')
                            print(*j, sep = ", ", end = '')
                            print("}", end = '')
                            print(" -> ", end = '')
                            of.write("{")
                            for h in range(len(j)):
                              of.write(str(j[h]))
                              if h != len(j) - 1:
                                of.write(", ")
                            of.write("}")
                            of.write(" -> ")
                        if len(i_tmp) == 1:
                            print(i_tmp[0], end = '')
                            of.write(str(i_tmp[0]))
                        else:
                            print("{", end = '')
                            print(*i_tmp, sep = ", ", end = '')
                           #print(i_tmp, end = '')
                            print("}", end = '')
                            of.write("{")
                            for h in range(len(i_tmp)):
                              of.write(str(i_tmp[h]))
                              if h != len(i_tmp) - 1:
                                of.write(", ")
                            of.write("}")
                        
                        print(f" ({i[1]}/{self.C[time - p - 2][k][1]}) ")
                        of.write(" (")
                        of.write(str(i[1]))
                        of.write("/")
                        of.write(str(self.C[time - p - 2][k][1]))
                        of.write(")\n")
        #print("--------------------")

    def find_combinations(self, b, i):
        a = list(itertools.combinations(b, i))#找出組合
        for j in range(len(a)):
            a[j] = list(a[j]) #change tuple to list
        return a

    def Final(self, i, of):
        stop = 0
        while stop != 1:
            #print(self.C[0])
            #print(f"i = {i}")
            C_befor_scan = self.find_combinations(self.C[0], i)#找出組合
            #print(C_befor_scan)
            C_befor_scan_2 = []
            for j in C_befor_scan:
                tmp = []
                for k in j:
                    tmp.append(k[0])
                C_befor_scan_2.append([tmp,0])#before scan
            #print(C_befor_scan_2)
            stop = 1
            for j in self.DataBase:
              for k in range(len(C_befor_scan_2)):
                if all(elem in j for elem in C_befor_scan_2[k][0]) :
                      stop = 0
                      C_befor_scan_2[k][1] = C_befor_scan_2[k][1] + 1 #after scan
            #print(C_befor_scan_2)
            if stop == 0:
                C_befor_scan_2 = self.CtoL(C_befor_scan_2)
                #print(C_befor_scan_2)
                self.Output(C_befor_scan_2, i, of)
                self.C.append(C_befor_scan_2)
                i = i + 1
    def run(self):
        self.Readfile()
        self.CreateC1()
        self.C[0] = self.CtoL(self.C[0])
        of = open(self.output_file_name, "a")
        self.Final(2, of)
        of.close()
        #print(*self.C, sep = "\n")

In [4]:
path = '/content/gdrive/MyDrive/'
input_file_name = path + 'input.txt'
output_file_name = path + 'output.txt'

min_support = int(input("Please input min support:"))
min_conf = int(input("Please input min conf:(%)"))


Please input min support:1600
Please input min conf:(%)66


In [ ]:
simulate = Apriori(min_support, min_conf, input_file_name, output_file_name)
Apriori.run(simulate)

36 -> 38 (2790/2936) 
36 -> 39 (2037/2936) 
38 -> 39 (10345/15596) 
110 -> 38 (2725/2794) 
170 -> 38 (3031/3099) 
41 -> 39 (11414/14945) 
48 -> 39 (29142/42135) 
89 -> 39 (2749/3837) 
170 -> 39 (2059/3099) 
225 -> 39 (2351/3257) 
310 -> 39 (1852/2594) 
89 -> 48 (2798/3837) 
{32, 41} -> 39 (2359/3196) 
{32, 48} -> 39 (5402/8034) 
{36, 38} -> 39 (1945/2790) 
{36, 39} -> 38 (1945/2037) 
36 -> {38, 39} (1945/2936) 
{38, 41} -> 39 (3051/3897) 
{38, 48} -> 39 (6102/7944) 
{39, 110} -> 38 (1740/1759) 
{38, 170} -> 39 (2019/3031) 
{39, 170} -> 38 (2019/2059) 
{41, 48} -> 39 (7366/9018) 
{48, 65} -> 39 (1797/2529) 
{39, 89} -> 48 (2125/2749) 
{48, 89} -> 39 (2125/2798) 
{32, 39, 41} -> 48 (1646/2359) 
{32, 41, 48} -> 39 (1646/2063) 
{38, 41, 48} -> 39 (1991/2374) 
